In [1]:
import sys
sys.path.insert(0, '../../src')
import harmoutil
import midigen
import numpy as np
from keras.models import load_model
import pickle as pkl

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Recreate model context (variables, mappings, ) (exact copy-paste of Label Harmonzation Model (depth=3).ipynb)

# Load data
raw_data = harmoutil.load_pickled_data("../../data/refined_data.pkl") # lists of (chord label, melody seqs) by sections
augmented_data = harmoutil.transpose_and_augment_data(raw_data)
data = [harmoutil.to_sevenths(section) for section in augmented_data]
data = [harmoutil.melody_to_octave_range(section) for section in data]


# Isolate meaningful data
def get_notes_by_chord(beats):
    return [note for beat in beats for note in beat]

def get_chords_by_section(section):
    return [chord_info[0] for chord_info in section]

chords_by_sections = [get_chords_by_section(section) for section in data]
chords = [chord_info[0] for section in data for chord_info in section]
unique_chords = sorted(list(set(chords)))

notes_by_chords = [get_notes_by_chord(chord_info[1]) for section in data for chord_info in section]
notes = [note for chord_notes in notes_by_chords for note in chord_notes]
unique_notes = sorted(list(set(notes)))


# Create categorical data mappings
note_to_int = dict([(c, i) for i, c in enumerate(unique_notes[1:])])
note_to_int[-1] = len(note_to_int)
note_to_int['<pad>'] = len(note_to_int)

int_to_note = dict([(k, v) for v, k in note_to_int.items()])

chord_to_int = dict([(c, i) for i, c in enumerate(unique_chords)])
chord_to_int['<bos>'] = len(chord_to_int)

int_to_chord = dict([(k, v) for v, k in chord_to_int.items()])



# Refine data that will actually be used
def check_if_augmented_major(section):
    section_chords = get_chords_by_section(section)
    for ch in section_chords:
        if "+j7" in ch:
            return True
    return False

def check_if_NC(section):
    section_chords = get_chords_by_section(section)
    for ch in section_chords:
        if "NC" in ch:
            return True
    return False

# Remove sections that involve augmented major chords (since not enough data to even allow StratifiedShuffleSplit)
section_data = [section for section in data if not check_if_augmented_major(section)]
print("Number of sections: {}\n".format(len(section_data)))

section_data = [section for section in section_data if not check_if_NC(section)]
print("Number of sections: {}\n".format(len(section_data)))

chords_by_sections = [get_chords_by_section(section) for section in section_data]
chords_data = [chord_info[0] for section in section_data for chord_info in section]
notes_by_chords = [get_notes_by_chord(chord_info[1]) for section in section_data for chord_info in section]



# Define numerical variables
n_chords = len(chord_to_int)
n_notes = len(note_to_int)
max_mel_len = max([len(mel) for mel in notes_by_chords])
chord_context_len = 7

print("Number of distinct melody notes: {}".format(n_notes))
print("Number of distinct chord labels: {}".format(n_chords))
print("Maximum length of melody sequences for one chord: {}".format(max_mel_len))
print("Number of past chords given as input: {}".format(chord_context_len))

Number of sections: 28836

Number of sections: 28416

Number of distinct melody notes: 14
Number of distinct chord labels: 194
Maximum length of melody sequences for one chord: 115
Number of past chords given as input: 7


In [3]:
# Load model
model_file = "../../harmonization_models_training/label_model/final/Label_depth3.h5"
model = load_model(model_file)

/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:304: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [4]:
# Tensor building functions for single entry
def build_mel_tensor(in_mel):
    padded_mel = in_mel + ['<pad>']*(max_mel_len - len(in_mel))
    X_mel = np.zeros((1, max_mel_len, n_notes), dtype='float32')
    for k, note in enumerate(padded_mel):
        X_mel[0, k, note_to_int[note]] = 1
    return X_mel

def build_ch_tensor(in_ch):
    X_ch = np.zeros((1, chord_context_len, n_chords), dtype='float32')
    for j, ch in enumerate(in_ch):
        X_ch[0, j, chord_to_int[ch]] = 1
    return X_ch

# Harmony predicting function updating tensors at every prediction
def predict_harmony(mel):
    harmony = []
    past_chords = ['<bos>']*(chord_context_len) # initial chord input
    
    for mel_notes in mel:
        input_mel = build_mel_tensor([harmoutil.root_index(note) for note in mel_notes])
        input_ch = build_ch_tensor(past_chords)
        next_pred = model.predict([input_mel, input_ch])
        next_chord = int_to_chord[np.argmax(next_pred)]
        past_chords = past_chords[1:] + [next_chord]
        harmony.append(next_chord)
    return harmony


# save harmony in same format as original harmony
def format_output_harmony(original, predicted):    
    harmo = []
    for bar in original:
        b = []
        for ch in bar:
            b.append(predicted[0])
            predicted = predicted[1:]
        harmo.append(b)
    return harmo

In [5]:
import os

# Generate harmony for each melody file
input_melody_dir = "../input_data/melody_files/"
truth_harmony_dir = "../input_data/truth_harmony_files/"
input_signature_dir = "../input_data/time_signature_files/"
output_harmony_dir = "label_harmony_files/"

for song_file in os.listdir(input_signature_dir):
    if not ".signature" in song_file:
        continue
    song_name = midigen.get_song_name(song_file)
    input_melody_file = input_melody_dir + song_name + ".melody"
    truth_harmony_file = truth_harmony_dir + song_name + ".chords"
    
    # load input melody and original harmony
    in_melody = midigen.read_melody_file_by_chord(input_melody_file)
    in_harmony = midigen.read_chords_file_by_bar(truth_harmony_file)

    # generate harmony
    predicted_harmony = predict_harmony(in_melody)        
    harmony = format_output_harmony(in_harmony, predicted_harmony)
    print(harmony)
    
    output_harmony_file = output_harmony_dir + song_name + ".pkl"
    with open(output_harmony_file, 'wb') as f:
        pkl.dump(harmony, f)

[['D-7'], ['Em7b5'], ['A7'], ['D-7'], ['Em7b5'], ['A7'], ['D-7'], ['Em7b5'], ['A7'], ['D-7'], ['Em7b5'], ['A7'], ['D-7'], ['Em7b5'], ['A7'], ['D-7']]
[['Gj7', 'A-7'], ['B-7', 'A-7'], ['Gj7', 'A-7'], ['B-7', 'E7'], ['A-7', 'A-7'], ['D7', 'G7'], ['C7'], ['Dbo', 'D7']]
[['G', 'E7'], ['A-7', 'D7'], ['B-7', 'E-7'], ['A-7'], ['D7', 'D-7'], ['G7', 'C7'], ['F7'], ['A-7', 'D7']]
[['C-7'], ['Dm7b5', 'G7'], ['C-7'], ['Dm7b5', 'G7'], ['C-7'], ['C-7', 'F7'], ['Bbj7'], ['Dm7b5', 'G7'], ['C-7'], ['F7', 'D7'], ['G7'], ['C-7'], ['F7'], ['F-7'], ['Bb7'], ['C-']]
[['G-7'], ['C7'], ['Em7b5'], ['Eb7'], ['D-7'], ['Bm7b5'], ['E7'], ['A-7'], ['G-7'], ['C7'], ['G-7'], ['C7'], ['G-7'], ['C7'], ['G-7'], ['C7']]
[['Eb-7'], ['Ab7'], ['Cm7b5'], ['F7'], ['Bb-7'], ['Gm7b5'], ['C7'], ['Ab-7']]
[['Ej7'], ['Fo7'], ['Db7'], ['Gb-7'], ['Go7'], ['Ej7'], ['Ebm7b5'], ['Ab7'], ['Aj7'], ['Abm7b5'], ['Db7'], ['Gb-7'], ['B-7'], ['Abm7b5'], ['Db7'], ['Gb-6']]
[['Dbj7', 'Bb7'], ['Eb-7', 'Ab7'], ['Dbj7', 'Bb7'], ['Eb-7', 'Ab7'], ['

In [ ]:
# Save formatted chord list 
